In [1]:
from pathlib import Path
import os
import sys

import pandas as pd
from unidecode import unidecode
import unicodedata
import numpy as np
import math as m
import re

## Funciones importantes 

In [2]:
# FUNCIONES IMPORTANTES
 #------------ Remueve acentos 
def remove_accents(input_str):
    # Normalizar el string a su forma combinada
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    # Filtrar y mantener solo los caracteres que no son diacríticos
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
# ----------- Crea data de zonas 
def Zone_data(Hoja,type):
    #excel = "diccionario_Benja/Diccionario_EMTP_DIgSILENT_BVega_v4.xlsx"
    excel = "diccionario_Benja/Zonas_DIgSILENT.xlsx"
    data= pd.read_excel(excel,sheet_name=Hoja)
    if type == "PV":
        columnas_deseadas = ['Name1', 'Name2','Zona DIgSILENT','Nombre DIgSILENT']  # Reemplaza con los nombres de las columnas que deseas
    elif type == "WP":
         columnas_deseadas = ['Name1','Zona DIgSILENT','Nombre DIgSILENT']     
    elif type == "SG":
        columnas_deseadas = ['Name1', 'Name2','Name3','Zona DIgSILENT','Nombre DIgSILENT']    
    elif type == "PMGD":
        columnas_deseadas = ['Name1', 'Name2','Zona DIgSILENT','Nombre DIgSILENT']  
    elif type == "CCSS":
        columnas_deseadas = ['Name1', 'Name2','Zona DIgSILENT','Nombre DIgSILENT']      
        
    # Filtra el DataFrame para que contenga solo las columnas deseadas
    dataframe_filtrado = data[columnas_deseadas]
    #return dataframe_filtrado.dropna()   
    return dataframe_filtrado   

def Transformación_MW_MVAR(cadena):
    #cadena_sin_mas = cadena[1:]
    #final= float(cadena_sin_mas)/(10**6)
    final= float(cadena)/(10**6)
    final=round(final,2)
    return (final)
def obtener_V_op(cadena):
    numeros = re.findall(r'[+-]?\d+\.\d+E[+-]?\d+', cadena)
    indices_pares = [i for i in range(len(numeros)) if i % 2 == 0]
    valores_pares = [numeros[i] for i in indices_pares]
    lista_V = [round(float(elemento),2) for elemento in valores_pares]
    if lista_V[0] == lista_V[1] == lista_V[2]:
        return lista_V[0]
    else:
        return "desbanceado"
# toma un el string que tiene "V+anglo y me entrega solo el "V y redondeado" 
def separar_numeros(s):
    numeros = re.findall(r'[+-]?\d+\.\d+E[+-]?\d+', s)
    if numeros:
        numero_1 = float(numeros[0])  # Convertir a float
        return round(numero_1, 2)
# obtengo cual es la tension nominal a partir del resultado
def analizar_numero(n):
    if 90 <= n + 20 <= 130 or 90 <= n - 20 <= 130:
        return 110
    elif 200 <= n + 20 <= 240 or 200 <= n - 20 <= 240:
        return 220
    elif 46 <= n + 20 <= 86 or 46 <= n - 20 <= 86:
        return 66
    elif 3 <= n + 10 <= 23.2 or 3 <= n - 10 <= 23.2:
        return 13.2    
    else:
        return None  # Si no cae en ninguna categoría


In [4]:
# Aqui coloco el nombre del archivo como string
#archivo =  "SEN 2025 VRE Peak 04072024 DBC 85 GFM New - H1_v4_lf.html"
#archivo = "SEN 2025 VRE Peak 85 VRE_g1_10 steps_lf.html"
#archivo = "SEN 2025 VRE Peak 85 VRE_g10_10 steps_lf.html"
archivo="SEN 2025 VRE Peak 85 VRE_Inicial_lf.html"
excel = "Datos_nom_v1.xlsx"

def Resultados_Load_Flow(archivo_html,excel):

    Datos_Nominal_extras = pd.read_excel(excel)
    df= pd.read_html(archivo_html)
    columnas = ['Tensión en Bornes [kV]','Potencia Activa [MW]', 'Potencia Reactiva [Mvar]']
    columnas2 = ['Name1','Name2', 'EMTP Load Flow Component']
    df2 = pd.DataFrame(columns=columnas2)
    
    # Crear el DataFrame vacío con las columnas definidas
    df_FINAL = pd.DataFrame(columns=columnas)
    df=pd.DataFrame(df[0])
    # Se procede a transformar el dataframe de html para que este legible 

    #obtengo el nombre las columnas y las reescribo
    fila_cero = df.iloc[0] 
    df.columns =fila_cero.to_list()
    #borro la fila 0 que tenia los nombres de las columnas
    df = df.iloc[1:]
    # reseteo los indices desde 0 y luego borro la primera columna indice
    df=pd.DataFrame(df.reset_index())
    lista_columnas = fila_cero.to_list()
    #  obtengo solo las 5 primeras importantes
    lista_columnas= lista_columnas[:5] 
    df2 = df[lista_columnas[:5]]
    # Crear una lista que solo contenga las centrales y no las cargas 
 
    valores_filtrar = ['PVbus', 'Slack', 'PQbus']
    df_filtrado = df2[df2['Type'].isin(valores_filtrar)]

    df_FINAL['Potencia Activa [MW]']=df_filtrado['P (W)'].apply(Transformación_MW_MVAR)
    df_FINAL['Potencia Reactiva [Mvar]']=df_filtrado['Q (VAR)'].apply(Transformación_MW_MVAR)
    df_FINAL["Tensión en Bornes [kV]"] = df_filtrado["Vabc (kVRMSLL,deg) phasor"].apply(obtener_V_op)
    df_FINAL['Tensión Nominal [kV]'] = df_FINAL["Tensión en Bornes [kV]"].apply(analizar_numero)
   

    #aqui la colummna device la expando para obtener 3 columnas de tal forma que coincida con lo anterior
    nuevo_df = df_filtrado['Device'].str.split('/', expand=True)
    nuevo_df.columns = ['Name1', 'Name2', 'NameLF',"Name4"]

    # Esto entrega la tabla de resultados de todas las centrales del HTML
    df_Resultados = pd.concat([nuevo_df, df_FINAL], axis=1)
    pd.set_option('display.max_rows', 140)

    #---------------------------- saco los acentos 
    df_Resultados['Name1'] = df_Resultados['Name1'].apply(lambda x: unidecode(x))
    df_Resultados['Name2'] = df_Resultados['Name2'].apply(lambda x: unidecode(x))
    df_Resultados = df_Resultados.drop(columns=['Name4']) #no tiene nada util

    ##------------------------Sobre escribo datos nominales------------------------------------
    
    N = len(df_Resultados)
    for fila in range(N):
        Name1= df_Resultados.at[fila,"Name1"]
        Name2= df_Resultados.at[fila,"Name2"]
        NameLF= df_Resultados.at[fila,"NameLF"]
        result = Datos_Nominal_extras[(Datos_Nominal_extras['Name1'] == Name1) &
                                 (Datos_Nominal_extras['Name2'] == Name2) & 
                                 (Datos_Nominal_extras['NameLF'] == NameLF)]
        if len(result)== 1:
            df_Resultados.at[fila,"Tensión Nominal [kV]"] =result["Tensión Nominal [kV]"]
        else: 
            continue 
    

    df_Resultados['Tensión [pu]'] = df_Resultados["Tensión en Bornes [kV]"]/df_Resultados['Tensión Nominal [kV]']
    # -------------------------- cambio orden la columna nombres 
    c=-1
    for name1 in df_Resultados['Name1']:
        c=c+1
        if "Central" in name1 or "Los_Quilos_Juncal" == name1 or "C_ANGOSTURA" == name1:
            df_Resultados.at[c,"Name3"]= name1 
            df_Resultados.at[c,"Name2"]= " " 
    #reordeno columnas 
    nuevo_orden = ['Name1','Name2','Name3','NameLF','Tensión en Bornes [kV]','Potencia Activa [MW]','Potencia Reactiva [Mvar]','Tensión Nominal [kV]','Tensión [pu]']
    #Reordenar las columnas
    df_reordenado = df_Resultados[nuevo_orden]
    return(df_reordenado)
  

Tabla_LF_GEN=Resultados_Load_Flow(archivo,excel)




Tabla_LF_GEN


C:\Users\fabian.medina\AppData\Local\Temp\ipykernel_30268\2366430498.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df_Resultados.at[fila,"Tensión Nominal [kV]"] =result["Tensión Nominal [kV]"]


,Name1,Name2,Name3,NameLF,Tensión en Bornes [kV],Potencia Activa [MW],Potencia Reactiva [Mvar],Tensión Nominal [kV],Tensión [pu]
0,Central_Ralco,,Central_Ralco,LF_RALC_U1,14.00,96.77,11.43,13.800,1.014493
1,Central_Alfalfal,,Central_Alfalfal,LF_ALFAL_U1,12.12,53.46,7.84,12.000,1.010000
2,Central_Alfalfal,,Central_Alfalfal,LF_ALFAL_U2,12.12,53.46,8.20,13.800,0.878261
3,Central_Alfalfal_II,,Central_Alfalfal_II,LF_ALFAL_II_U1,15.60,39.20,25.79,15.000,1.040000
4,Central_Alfalfal_II,,Central_Alfalfal_II,LF_ALFAL_II_U2,15.60,39.20,25.79,15.000,1.040000
5,Central_Angamos,,Central_Angamos,LF_ANG_U2,18.00,100.00,-27.44,18.000,1.000000
6,C_ANGOSTURA,,C_ANGOSTURA,LF2,13.80,112.30,-5.10,13.800,1.000000
7,Central_Canutillar,,Central_Canutillar,LF_CANU_U1,14.20,25.00,5.81,13.800,1.028986
8,Central_Canutillar,,Central_Canutillar,LF_CANU_U2,14.20,25.00,5.81,13.800,1.028986
9,Central_Colbun,,Central_Colbun,LF_COLB_U1,14.20,100.00,13.84,13.800,1.028986


## Obteniendo DATA

In [5]:
# version FABI
carpeta = "Data/2030/Data_2030_i0"
archivos = os.listdir(carpeta)
print(archivos)

df_allSG_csv = [s for s in archivos if "data_allSG" in s]
df_lfToSG_csv = [s for s in archivos if "data_lfToSG" in s]
df_PV_csv = [s for s in archivos if "data_PV" in s]
df_WP_csv = [s for s in archivos if "data_WP" in s]


df_allSG  = pd.read_csv(carpeta+"/"+df_allSG_csv[0])
df_lfToSG  = pd.read_csv(carpeta+"/"+df_lfToSG_csv[0])
df_PV  = pd.read_csv(carpeta+"/"+df_PV_csv[0],encoding='latin1')
df_WP  = pd.read_csv(carpeta+"/"+df_WP_csv[0])

#data status
df_pv_csv = [s for s in archivos if "uc_pv" in s]
df_wp_csv = [s for s in archivos if "uc_wp" in s]
df_sg_csv = [s for s in archivos if "uc_sg" in s]

df_pv  = pd.read_csv(carpeta+"/"+df_pv_csv[0],encoding='latin1')
df_wp  = pd.read_csv(carpeta+"/"+df_wp_csv[0],encoding='latin1')
df_sg  = pd.read_csv(carpeta+"/"+df_sg_csv[0],encoding='latin1')


#df_PV['Name_PV'] = df_PV['Name_PV'].apply(lambda x: unidecode(x))
## ------------------- correcciones 
## agrego ultimo CCSS al status
df1 = pd.DataFrame({
    'Name1': ["CCSS_TOCOPILLA_ENGIE"],
    'Name2': ["-"],
    'Name3': ["-"],
    "Status1":	["1"],
    "Status2":	["-"],
    "Status3":	["-"],
    "StatusFinal": 	["1"]
})

df_sg = pd.concat([df_sg, df1], axis=0, ignore_index=True)
df_sg

['data_allSG_SEN 2026_Redespacho_NG.csv', 'data_lfToSG_SEN 2026_Redespacho_NG.csv', 'data_LF_SEN 2026_Redespacho_NG.csv', 'data_PV_SEN 2026_Redespacho_NG.csv', 'data_WP_SEN 2026_Redespacho_NG.csv', 'uc_converter_SEN 2026_Redespacho_NG.csv', 'uc_pv_SEN 2026_Redespacho_NG.csv', 'uc_sg_SEN 2026_Redespacho_NG.csv', 'uc_wp_SEN 2026_Redespacho_NG.csv']


,Name1,Name2,Name3,Status1,Status2,Status3,StatusFinal
0,Central_Colbun,Colbun_U1,Colbun_U1,1,1,1,1
1,Central_Colbun,Colbun_U2,Colbun_U2,1,1,1,1
2,Central_Canutillar,Canutillar_U1,Canutillar_U1,1,1,1,1
3,Central_Canutillar,Canutillar_U2,Canutillar_U2,1,1,1,1
4,Central_Pangue,Pangue_U1,Pangue_U1,1,1,1,1
...,...,...,...,...,...,...,...
151,HP_LOS_CONDORES_OG,-,-,0,-,-,0
152,HP_NUBLE_OG,-,-,0,-,-,0
153,TER_DONA_LUZMA,-,-,0,-,-,0
154,HP_LOS_LAGOS_OG,-,-,0,-,-,0


In [6]:
##------------- zonas 
zone_data_WP=Zone_data("Static_WP","WP")
zone_data_PV=Zone_data("Static_PV","PV")
zone_data_SG=Zone_data("SG_UC","SG")

zone_data_PMGD=Zone_data("PMGD","PMGD")
zone_data_CCSS=Zone_data("CCSS","CCSS")
zone_data_CCSS


,Name1,Name2,Zona DIgSILENT,Nombre DIgSILENT
0,CCSS_AMARIA_ALUPAR,NaN,00-Norte Grande,NaN
1,CCSS_AMARIA_TRANSELEC,NaN,00-Norte Grande,NaN
2,CCSS_TOCOPILLA_ENGIE,NaN,00-Norte Grande,NaN
3,CCSS_LIKANANTAI_TRANSELEC,NaN,00-Norte Grande,NaN
4,CCSS_ILLAPA_ALUPAR,NaN,01-Atacama,NaN


In [13]:
datos_PV = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_PMGD = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_PV_OG = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))                                                                      


X = "PV"
def datos_PFV(datos_X,df_x,df_X,X,zones):  
    datos_X["Name1"]=df_x["Name1"]
    datos_X["Name2"]=df_x["Name2"]
    datos_X["EMTP Status"] = df_x["StatusFinal"]


    if X == "PV":
        Nombre = "Name2"
        Eliminados = ["PMGD","OG"]  
        Tipo = ["Parque Fotovoltaico"]      
        for i in Eliminados:
            datos_X = datos_X[~datos_X['Name1'].str.contains(i, na=False)]
        datos_X = datos_X.reset_index(drop=True)
        datos_X["type"]= len(datos_X)*Tipo    
        datos_X['Name1'] = datos_X['Name1'].apply(lambda x: unidecode(x))
        datos_X['Name2'] = datos_X['Name2'].apply(lambda x: unidecode(x))    
    
    elif X == "PMGD":
        Nombre = "Name1"    
        Tipo = ["PMGD"]
        datos_X = datos_X[datos_X['Name1'].str.contains('PMGD', na=False)]
        datos_X = datos_X.reset_index(drop=True)
        datos_X["Type"] = len(datos_X)*Tipo

    elif X == "PV_OG":
         Nombre = "Name1"
         datos_X = datos_X[datos_X['Name1'].str.contains('OG', na=False)]
         datos_X = datos_X.reset_index(drop=True)
         datos_X["Type"] = len(datos_X)*["Parque Fotovoltaico"]


    c=-1
    for name1 in datos_X[Nombre]:
        c=c+1
        indice= df_X.index[df_X['Name_PV'] ==name1].tolist()[0] 
        Spv= df_X.at[indice+7,"Value"]
        Vnom = df_X.at[indice+3,"Value"]
        Npv =df_X.at[indice,"Value"]
        Npv_serv = df_X.at[indice+17,"Value"]
        Snom = Spv*Npv
        Sbase = Spv*Npv_serv
        Qset = df_X.at[indice+19,"Value"]
        Pset = df_X.at[indice+21,"Value"]
    
    #--------------------------------------------------------------
        datos_X.at[c,'Nº_unid']= Npv
        datos_X.at[c,'Nº_unid_serv']= Npv_serv
        datos_X.at[c,'Tensión Nominal [kV]']=Vnom
        datos_X.at[c,"Potencia Nominal [MVA]"] = Snom
        datos_X.at[c,"Sbase [MVA]"] = Sbase
        datos_X.at[c,"P_set [MW]"] = Pset*Sbase
        datos_X.at[c,"Q_set [Mvar]"] = Qset*Sbase
        N = len(zones)         
    for i in range(0,N):
        c=-1
        for name1 in datos_X["Name1"]:   
            c=c+1
            if name1 == zones.iloc[i,0]:
                    datos_X.at[c,"Página PowerFactory"] = zones.iloc[i,2]
                    datos_X.at[c,'Nombre PowerFactory'] = zones.iloc[i,3]
            else:
                continue  
        
    return datos_X 
    

    
datos_PV = datos_PFV(datos_PV,df_pv,df_PV,"PV",zone_data_PV)
    
datos_PMGD = datos_PFV(datos_PMGD,df_pv,df_PV,"PMGD",zone_data_PMGD)
datos_PV_OG= datos_PFV(datos_PV_OG,df_pv,df_PV,"PV_OG",zone_data_PV)
datos_PV_OG

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PFV_Don_Humberto_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,03-Chilquinta-Aconcagua,NaN,NaN,NaN,NaN,141.0,59.9955,0.0,NaN,110.0,141.0,141.0,141.0
1,PFV_Central_DDA_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,293.0,110.168,0.0,NaN,220.0,293.0,293.0,293.0
2,PFV_Gabriela_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,261.0,220.545,0.0,NaN,220.0,261.0,261.0,261.0
3,PFV_Andes_III_Etapa1_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,176.0,128.48,0.0,NaN,220.0,176.0,176.0,176.0
4,PFV_Victor_Jara_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,240.0,200.16,0.0,NaN,220.0,240.0,240.0,240.0
5,PFV_Las_Salinas_Etapa2_OG,-,NaN,NaN,Parque Fotovoltaico,0,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,36.0,0.036,0.0,NaN,220.0,36.0,36.0,36.0
6,PFV_Las_Salinas_Etapa3_OG,-,NaN,NaN,Parque Fotovoltaico,0,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,127.0,0.127,0.0,NaN,220.0,127.0,127.0,127.0
7,PFV_Las_Salinas_Etapa4_OG,-,NaN,NaN,Parque Fotovoltaico,0,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,97.0,0.097,0.0,NaN,220.0,97.0,97.0,97.0
8,PFV_Las_Salinas_Etapa5_OG,-,NaN,NaN,Parque Fotovoltaico,0,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,31.0,0.031,0.0,NaN,220.0,31.0,31.0,31.0
9,PFV_Gran_Teno_OG,-,NaN,NaN,Parque Fotovoltaico,0,NaN,07-Sistema154 - 66 kV (Centro),NaN,NaN,NaN,NaN,241.0,0.241,0.0,NaN,220.0,241.0,241.0,241.0


# Data WP


In [16]:
datos_WP = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_WP["Name1"] =df_wp["Name1"]
datos_WP["Name2"] =df_wp["Name2"]   
datos_WP["EMTP Status"] = df_wp["StatusFinal"]
datos_WP["Type"] = len(datos_WP)*["Parque Eólico"]

c=-1
for name1 in datos_WP["Name1"]:
    c=c+1
    indice= df_WP.index[df_WP['Name_WP'] ==name1].tolist()[0]

    Swp= df_WP.at[indice+7,"Value"]
    Vnom = df_WP.at[indice+3,"Value"]
    Nwp =df_WP.at[indice,"Value"]
    
    Snom = Swp*Nwp
    
    
    if name1 != "PE_Taltal":
        Pset = df_WP.at[indice+20,"Value"]
        Qset = df_WP.at[indice+19,"Value"]
        Nwp_serv = df_WP.at[indice+17,"Value"]
        Sbase = Swp*Nwp_serv
    else :
        Pset =0
        # Qset = df_WP.at[indice+18,"Value"]   
        Nwp_serv = df_WP.at[indice+16,"Value"]
        Sbase = Swp*Nwp_serv  

    #--------------------------------------------------------------
    datos_WP.at[c,'Nº_unid']= Nwp
    datos_WP.at[c,'Nº_unid_serv']= Nwp_serv
    datos_WP.at[c,'Tensión Nominal [kV]']=Vnom
    datos_WP.at[c,"Potencia Nominal [MVA]"] = Snom
    datos_WP.at[c,"Sbase [MVA]"] = Sbase
    datos_WP.at[c,"P_set [MW]"] = Pset*Sbase
    datos_WP.at[c,"Q_set [Mvar]"] = Qset*Sbase

#----------------------------------------------------------------------------
c=-1 
N = len(zone_data_WP)
Nstatus= len(df_wp)
for name in datos_WP["Name1"]:
    c=c+1
    #agrego zona y pagina PF
    for i in range(0,N):
        if name == zone_data_WP.iloc[i,0]:
            datos_WP.at[c,"Página PowerFactory"] = zone_data_WP.iloc[i,1]
            datos_WP.at[c,'Nombre PowerFactory'] = zone_data_WP.iloc[i,2]
        else:
            continue                    

datos_WP      

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PE_Aurora,-,NaN,PE AURORA (U1-U43),Parque Eólico,1,NaN,10-Araucanía,NaN,NaN,NaN,NaN,123.358,115.95652,-12.459158,NaN,220.0,123.358,74.0,74.0
1,PE_SanGabriel,-,NaN,PE SAN GABRIEL (U1-U61),Parque Eólico,1,NaN,10-Araucanía,NaN,NaN,NaN,NaN,183.37,153.11395,-0.495099,NaN,220.0,183.37,110.0,110.0
2,PE_MallecoNorte,-,NaN,PE Malleco Norte,Parque Eólico,1,NaN,10-Araucanía,NaN,NaN,NaN,NaN,138.361,130.05934,-7.665199,NaN,220.0,138.361,83.0,83.0
3,PE_MallecoSur,-,NaN,PE Malleco Sur,Parque Eólico,1,NaN,10-Araucanía,NaN,NaN,NaN,NaN,135.027,82.6832,-18.53704,NaN,220.0,135.027,81.0,81.0
4,PE_Los_Olmos,-,NaN,PE LOS OLMOS,Parque Eólico,1,NaN,10-Araucanía,NaN,NaN,NaN,NaN,108.355,105.10435,-11.442288,NaN,220.0,108.355,65.0,65.0
5,PE_Tolpan_Sur,-,NaN,PE Tolpan Sur,Parque Eólico,1,NaN,10-Araucanía,NaN,NaN,NaN,NaN,83.35,73.5147,-4.61759,NaN,220.0,83.35,50.0,50.0
6,PE_Lomas_de_Duqueco,-,NaN,PE LOMAS DE DUQUECO,Parque Eólico,1,NaN,08-Charrúa,NaN,NaN,NaN,NaN,61.679,32.30646,-0.20004,NaN,66.0,61.679,37.0,37.0
7,PE_El_Arrayan,-,NaN,PE EL ARRAYAN (U1-U50),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,116.69,26.0052,2.10042,NaN,220.0,116.69,70.0,70.0
8,PE_Pta_Palmeras,-,NaN,PE PUNTA PALMERAS (U1-U15),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,45.009,37.402479,-6.75135,NaN,220.0,45.009,27.0,27.0
9,PE_Talinay_O_P,-,NaN,PE TALINAY ORIENTE (U1-U30),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,161.699,67.3468,-0.60012,NaN,220.0,161.699,97.0,97.0


In [ ]:
datos_SG = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_SG_OG = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
def datos_SG(datos_X,df_x,df_X,X,zones):  
    datos_X["Name1"]=df_x["Name1"]
    datos_X["Name2"]=df_x["Name2"]
    datos_X["EMTP Status"] = df_x["StatusFinal"]                                   

    if X == "SG":
        Nombre = "Name2"
        Eliminados = ["OG","CCSS","SM3"]   
        for i in Eliminados:
            datos_X = datos_X[~datos_X['Name1'].str.contains(i, na=False)]   
        datos_X = datos_X.reset_index(drop=True)
        datos_X["Type"] = len(datos_X)*["Generador Sincrónico"]

    for name3 in datos_X["Name3"]:  
        c=c+1
        try:
            indice = df_lfToSG.index[df_lfToSG['AssociateSG'] == name3].tolist()[0]
            resultado = "sí"
        except IndexError:
            resultado = "no"
        
        datos_X.at[c,"EMTP Load Flow Component"]= df_lfToSG.at[indice,"Name_of_LF"] #Lf
        datos_X.at[c,"Tipo de Barra Flujo de Carga"]= df_lfToSG.at[indice,"Value"] #tipo de barra
        datos_X.at[c,"V_set [kV]"]= df_lfToSG.at[indice+1,"Value"]   # V_set
        datos_X.at[c,"P_set [MW]"]= df_lfToSG.at[indice+2,"Value"]   # P_set
        datos_SG.at[c,"Q_set [Mvar]"]= df_lfToSG.at[indice+3,"Value"] # Q_se  

# -------- Agrego data nominal 
    N = len(df_allSG)
    c=-1
    for name3 in datos_X["Name3"]:
        c=c+1
        for i in range(0,N):
            if name3 == df_allSG.at[i,"Name_of_SG"]:
                datos_X.at[c,"Tensión Nominal [kV]"]= df_allSG.at[i+3,"Value"]
                datos_X.at[c,"Potencia Nominal [MVA]"]= df_allSG.at[i+4,"Value"]
                datos_X.at[c,"Sbase [MVA]"] = df_allSG.at[i+4,"Value"]   # V_set   
# agrego data PF
    N = len(zone_data_SG)
    c=-1
    for name3 in datos_X["Name3"]:
        c=c+1
        #agrego zona y pagina PF
    for i in range(0,N):
        if name3 == zone_data_SG.iloc[i,2]:
            datos_X.at[c,"Página PowerFactory"] = zone_data_SG.iloc[i,3]
            datos_X.at[c,'Nombre PowerFactory'] = zone_data_SG.iloc[i,4]
        else:  
            continue    
    Nsg = len(datos_SG)
    datos_X["Type"] = ["Generador Sincrónico"]*Nsg          
    datos_X["Nº_unid_serv"] = ["1"]*Nsg   
    datos_X["Nº_unid"] = ["1"]*Nsg         
    #pd.set_option('display.max_rows', 151)
    return(datos_X)                                                                                     

# Datos SG

In [17]:
datos_SG = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_SG["Name1"] =df_sg["Name1"]
datos_SG["Name2"] =df_sg["Name2"]
datos_SG["Name3"] =df_sg["Name3"]
datos_SG["EMTP Status"] = df_sg["StatusFinal"]
##----------- elimino los OG

datos_SG = datos_SG[~datos_SG['Name1'].str.contains('OG', na=False)]
datos_SG = datos_SG[~datos_SG['Name1'].str.contains('CCSS', na=False)]
datos_SG = datos_SG[~datos_SG['Name1'].str.contains('SM3', na=False)]
datos_SG = datos_SG.reset_index(drop=True)
datos_SG["Type"] = len(datos_SG)*["Generador Sincrónico"]


##-------------------------------------------------------------------

pd.set_option('display.max_rows', 151)


c=-1
l_aux= []  
for name3 in datos_SG["Name3"]:
    l_aux.append(name3)
    
    c=c+1
    try:
        indice = df_lfToSG.index[df_lfToSG['AssociateSG'] == name3].tolist()[0]
        resultado = "sí"
    except IndexError:
        resultado = "no"
        print(name3)
    datos_SG.at[c,"EMTP Load Flow Component"]= df_lfToSG.at[indice,"Name_of_LF"] #Lf
    datos_SG.at[c,"Tipo de Barra Flujo de Carga"]= df_lfToSG.at[indice,"Value"] #tipo de barra
    datos_SG.at[c,"V_set [kV]"]= df_lfToSG.at[indice+1,"Value"]   # V_set
    datos_SG.at[c,"P_set [MW]"]= df_lfToSG.at[indice+2,"Value"]   # P_set
    datos_SG.at[c,"Q_set [Mvar]"]= df_lfToSG.at[indice+3,"Value"] # Q_set
     
   

 
 
 

# agrego data nominal    

N = len(df_allSG)
c=-1
for name3 in datos_SG["Name3"]:
    c=c+1
    for i in range(0,N):
        if name3 == df_allSG.at[i,"Name_of_SG"]:
             datos_SG.at[c,"Tensión Nominal [kV]"]= df_allSG.at[i+3,"Value"]
             datos_SG.at[c,"Potencia Nominal [MVA]"]= df_allSG.at[i+4,"Value"]
             datos_SG.at[c,"Sbase [MVA]"] = df_allSG.at[i+4,"Value"]   # V_set
            
# agrego data PF
N = len(zone_data_SG)
c=-1
for name3 in datos_SG["Name3"]:
    c=c+1
    #agrego zona y pagina PF
    for i in range(0,N):
        if name3 == zone_data_SG.iloc[i,2]:
            datos_SG.at[c,"Página PowerFactory"] = zone_data_SG.iloc[i,3]
            datos_SG.at[c,'Nombre PowerFactory'] = zone_data_SG.iloc[i,4]
        else:
            continue
Nsg = len(datos_SG)
datos_SG["Type"] = ["Generador Sincrónico"]*Nsg          
datos_SG["Nº_unid_serv"] = ["1"]*Nsg   
datos_SG["Nº_unid"] = ["1"]*Nsg         
#pd.set_option('display.max_rows', 151)
datos_SG     


    


-


,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,Central_Colbun,Colbun_U1,Colbun_U1,HE COLBUN U1,Generador Sincrónico,1,LF_COLB_U1,05-Colbún,NaN,NaN,NaN,PV,220.0,100,480,14.2,13.8,220.0,1,1
1,Central_Colbun,Colbun_U2,Colbun_U2,HE COLBUN U2,Generador Sincrónico,1,LF_COLB_U2,05-Colbún,NaN,NaN,NaN,PV,220.0,100,23.8,14.2,13.8,220.0,1,1
2,Central_Canutillar,Canutillar_U1,Canutillar_U1,HE CANUTILLAR U1,Generador Sincrónico,1,LF_CANU_U1,10-Araucanía,NaN,NaN,NaN,PV,70.0,25,480,14.2,13.8,70.0,1,1
3,Central_Canutillar,Canutillar_U2,Canutillar_U2,HE CANUTILLAR U21,Generador Sincrónico,1,LF_CANU_U2,10-Araucanía,NaN,NaN,NaN,PV,70.0,25,480,14.2,13.8,70.0,1,1
4,Central_Pangue,Pangue_U1,Pangue_U1,HE PANGUE U1,Generador Sincrónico,1,LF_PANG_U1,08-Charrúa,NaN,NaN,NaN,PV,240.0,73.9,480,13.5,13.8,240.0,1,1
5,Central_Pangue,Pangue_U2,Pangue_U2,HE PANGUE U2,Generador Sincrónico,0,LF_PANG_U2,08-Charrúa,NaN,NaN,NaN,PV,240.0,182.4,480,13.524,13.8,240.0,1,1
6,Central_Antuco,Antuco_U2,HE_Antuco_U1,NaN,Generador Sincrónico,1,LF_ANTU_U2,08-Charrúa,NaN,NaN,NaN,PV,160.0,80.8,480,13.2,13.8,160.0,1,1
7,Central_Antuco,Antuco_U1,Antuco_U1,HE ANTUCO U1,Generador Sincrónico,0,LF_ANTU_U1,08-Charrúa,NaN,NaN,NaN,PV,160.0,80,480,13.938,13.8,160.0,1,1
8,Central_Ralco,Ralco_U1,Ralco_U1,HE RALCO U1,Generador Sincrónico,1,LF_RALC_U1,08-Charrúa,NaN,NaN,NaN,Slack,402.0,186.5,480,14,13.8,402.0,1,1
9,Central_Ralco,Ralco_U2,Ralco_U2,HE RALCO U2,Generador Sincrónico,0,LF_RALC_U2,08-Charrúa,NaN,NaN,NaN,PV,402.0,120,480,13.8,13.8,402.0,1,1


# DATOS SG OG

In [18]:
datos_SG_OG = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_SG_OG["Name1"] =df_sg["Name1"]
datos_SG_OG["Name2"] =df_sg["Name2"]
datos_SG_OG["Name3"] =df_sg["Name3"]
datos_SG_OG["EMTP Status"] = df_sg["StatusFinal"]
##----------- elimino los OG
datos_SG_OG = datos_SG_OG[~datos_SG_OG['Name1'].str.contains('CCSS', na=False)]
datos_SG = datos_SG[~datos_SG['Name1'].str.contains('SM3', na=False)]
datos_SG_OG = datos_SG_OG[datos_SG_OG['Name1'].str.contains('OG', na=False)]


datos_SG_OG = datos_SG_OG.reset_index(drop=True)
datos_SG_OG["Type"] = len(datos_SG_OG)*["Generador Sincrónico"]

df_lfToSGOG = pd.read_excel('Plantas.xlsx',sheet_name="SGOG" )
##-------------------------------------------------------------------

c=-1
l_aux= []  
for name1 in datos_SG_OG["Name1"]:
    l_aux.append(name3)
    
    c=c+1
    try:
        indice = df_lfToSGOG.index[df_lfToSGOG['AssociateSG'] == name1].tolist()[0]
        resultado = "sí"
    except IndexError:
        resultado = "no"
        print(name1)
    datos_SG_OG.at[c,"EMTP Load Flow Component"]= df_lfToSG.at[indice,"Name_of_LF"] #Lf
    datos_SG_OG.at[c,"Tipo de Barra Flujo de Carga"]= df_lfToSG.at[indice,"Value"] #tipo de barra
    datos_SG_OG.at[c,"V_set [kV]"]= df_lfToSG.at[indice+1,"Value"]   # V_set
    datos_SG_OG.at[c,"P_set [MW]"]= df_lfToSG.at[indice+2,"Value"]   # P_set
    datos_SG_OG.at[c,"Q_set [Mvar]"]= df_lfToSG.at[indice+3,"Value"] # Q_set
     
# agrego data nominal    

N = len(df_allSG)
c=-1
for name1 in datos_SG_OG["Name1"]:
    c=c+1
    for i in range(0,N):
        if name1 == df_allSG.at[i,"Name_of_SG"]:
             datos_SG_OG.at[c,"Tensión Nominal [kV]"]= df_allSG.at[i+3,"Value"]
             datos_SG_OG.at[c,"Potencia Nominal [MVA]"]= df_allSG.at[i+4,"Value"]
             datos_SG_OG.at[c,"Sbase [MVA]"] = df_allSG.at[i+4,"Value"]   # V_set
            
# agrego data PF
N = len(zone_data_SG)
c=-1
for name1 in datos_SG_OG["Name1"]:
    c=c+1
    #agrego zona y pagina PF
    for i in range(0,N):
        if name3 == zone_data_SG.iloc[i,2]:
            datos_SG_OG.at[c,"Página PowerFactory"] = zone_data_SG.iloc[i,3]
            datos_SG_OG.at[c,'Nombre PowerFactory'] = zone_data_SG.iloc[i,4]
        else:
            continue
Nsg = len(datos_SG_OG)
datos_SG_OG["Type"] = ["Generador Sincrónico"]*Nsg          
datos_SG_OG["Nº_unid_serv"] = ["1"]*Nsg   
datos_SG_OG["Nº_unid"] = ["1"]*Nsg         
#pd.set_option('display.max_rows', 151)   

pd.set_option('display.max_rows', 151)




datos_SG_OG      

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,HP_TRUPAN_OG,-,-,NaN,Generador Sincrónico,0,LF_COLB_U1,NaN,NaN,NaN,NaN,PV,20.0,100,480,14.2,13.8,20.0,1,1
1,HP_DOS_VALLES_AMP_OG,-,-,NaN,Generador Sincrónico,0,LF_COLB_U2,NaN,NaN,NaN,NaN,PV,5.0,100,23.8,14.2,13.8,5.0,1,1
2,HP_LOS_CONDORES_OG,-,-,NaN,Generador Sincrónico,0,LF_LGUINDOS_U1,NaN,NaN,NaN,NaN,PV,162.0,0,480,15,13.8,162.0,1,1
3,HP_NUBLE_OG,-,-,NaN,Generador Sincrónico,0,LF_LGUINDOS_U2,NaN,NaN,NaN,NaN,PV,136.0,0,480,15,13.8,136.0,1,1
4,HP_LOS_LAGOS_OG,-,-,NaN,Generador Sincrónico,0,LF_SFeE_U1_U1,NaN,NaN,NaN,NaN,PV,54.0,20,480,13.2,13.8,54.0,1,1


# Datos CCSS

In [19]:
datos_CCSS = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_CCSS["Name1"] =df_sg["Name1"]
datos_CCSS["Name2"] =df_sg["Name2"]
datos_CCSS["Name3"] =df_sg["Name3"]
datos_CCSS["EMTP Status"] = df_sg["StatusFinal"]
##----------- elimino los OG
#datos_CCSS2= datos_CCSS[datos_CCSS['Name1'].str.contains('SM3', na=False)]
datos_CCSS = datos_CCSS[datos_CCSS['Name1'].str.contains('CCSS', na=False)]
#datos_CCSS = pd.concat([datos_CCSS, datos_CCSS2], axis=0, ignore_index=True)

datos_CCSS = datos_CCSS.reset_index(drop=True)
datos_CCSS["Type"] = len(datos_CCSS)*["Generador Sincrónico"]

df_lfToSGCCSS = pd.read_excel('Plantas.xlsx',sheet_name='CCSS')
##-------------------------------------------------------------------



c=-1
l_aux= []  
for name1 in datos_CCSS["Name1"]:
    l_aux.append(name3)
    
    c=c+1
    try:
        indice = df_lfToSGCCSS.index[df_lfToSGCCSS ['AssociateSG'] == name1].tolist()[0]
        resultado = "sí"
    except IndexError:
        resultado = "no"
        print(name1)
    datos_CCSS.at[c,"EMTP Load Flow Component"]= df_lfToSGCCSS.at[indice,"Name_of_LF"] #Lf
    datos_CCSS.at[c,"Tipo de Barra Flujo de Carga"]= df_lfToSG.at[indice,"Value"] #tipo de barra
    datos_CCSS.at[c,"V_set [kV]"]= df_lfToSGCCSS.at[indice+1,"Value"]   # V_set
    datos_CCSS.at[c,"P_set [MW]"]= df_lfToSGCCSS.at[indice+2,"Value"]   # P_set
    datos_CCSS.at[c,"Q_set [Mvar]"]= df_lfToSGCCSS.at[indice+3,"Value"] # Q_set
# agrego data nominal    

N = len(df_allSG)
c=-1
for name1 in datos_CCSS["Name1"]:
    c=c+1
    for i in range(0,N):
        if name1 == df_allSG.at[i,"Name_of_SG"]:
             datos_CCSS.at[c,"Tensión Nominal [kV]"]= df_allSG.at[i+3,"Value"]
             datos_CCSS.at[c,"Potencia Nominal [MVA]"]= df_allSG.at[i+4,"Value"]
             datos_CCSS.at[c,"Sbase [MVA]"] = df_allSG.at[i+4,"Value"]   # V_set
                
            
# agrego data PF
N = len(zone_data_CCSS)
c=-1
for name1 in datos_CCSS["Name1"]:
    c=c+1
    #agrego zona y pagina PF
    for i in range(0,N):
        if name1 == zone_data_CCSS.iloc[i,0] :
            print(name1)
            datos_CCSS.at[c,"Página PowerFactory"] = zone_data_CCSS.iloc[i,2]
            datos_CCSS.at[c,'Nombre PowerFactory'] = zone_data_CCSS.iloc[i,3]
        else:
            continue     
Nsg = len(datos_CCSS)
datos_CCSS["Nº_unid_serv"] = ["1"]*Nsg   
datos_CCSS["Nº_unid"] = ["1"]*Nsg         
#pd.set_option('display.max_rows', 151)   
        
datos_CCSS

CCSS_TOCOPILLA_ENGIE
CCSS_AMARIA_ALUPAR
CCSS_AMARIA_TRANSELEC
CCSS_LIKANANTAI_TRANSELEC
CCSS_ILLAPA_ALUPAR
CCSS_TOCOPILLA_ENGIE


,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,CCSS_AMARIA_ALUPAR,-,-,NaN,Generador Sincrónico,0,LF5,00-Norte Grande,NaN,NaN,NaN,PV,360.0,0,0,15,15.0,360.0,1,1
1,CCSS_AMARIA_TRANSELEC,-,-,NaN,Generador Sincrónico,0,LF7,00-Norte Grande,NaN,NaN,NaN,PV,92.0,0,0,15,15.0,92.0,1,1
2,CCSS_LIKANANTAI_TRANSELEC,-,-,NaN,Generador Sincrónico,0,LF6,00-Norte Grande,NaN,NaN,NaN,PV,92.0,0,0,15,15.0,92.0,1,1
3,CCSS_ILLAPA_ALUPAR,-,-,NaN,Generador Sincrónico,0,LF4,01-Atacama,NaN,NaN,NaN,PV,270.0,0,0,15,15.0,270.0,1,1
4,CCSS_TOCOPILLA_ENGIE,-,-,NaN,Generador Sincrónico,1,LF4,00-Norte Grande,NaN,NaN,NaN,PV,147.1,0,0,15,13.8,147.1,1,1


# Uniendo Data

In [20]:
df_concatenado = pd.concat([datos_PV, datos_WP, datos_SG,datos_PMGD,datos_PV_OG,datos_SG_OG,datos_CCSS ], axis=0, ignore_index=True)
df_concatenado['P_set [MW]'] = df_concatenado['P_set [MW]'].astype(float)
df_concatenado['Q_set [Mvar]'] = df_concatenado['Q_set [Mvar]'].astype(float)


df_concatenado
#df_concatenado

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PFV_Capricornio,PFV_Capricornio_new,NaN,PFV CAPRICORNIO,Parque Fotovoltaico,1,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,95.019,95.019000,-21.673834,NaN,110.0,95.019,57.0,57.0
1,PFV_Quilapilun,PFV_Quilapilun,NaN,PFV QUILAPILUN,Parque Fotovoltaico,1,NaN,03-Chilquinta-Aconcagua,NaN,NaN,NaN,NaN,110.022,99.219840,-1.500300,NaN,220.0,110.022,66.0,66.0
2,PFV_Dona_Carmen,PFV_Dona_Carmen,NaN,PFV DOÑA CARMEN SOLAR,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,35.007,27.938920,0.000000,NaN,220.0,35.007,21.0,21.0
3,PFV_EL_PELICANO,PFV_EL_PELICANO,NaN,NaN,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,110.0022,67.871357,-16.104322,NaN,220.0,110.0022,66.0,66.0
4,PFV_LA_HUELLA,PFV_LA_HUELLA,NaN,PFV LA HUELLA,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,94.38,58.232460,-24.066900,NaN,220.0,94.38,26.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,CCSS_AMARIA_ALUPAR,-,-,NaN,Generador Sincrónico,0,LF5,00-Norte Grande,NaN,NaN,NaN,PV,360.0,0.000000,0.000000,15,15.0,360.0,1,1
320,CCSS_AMARIA_TRANSELEC,-,-,NaN,Generador Sincrónico,0,LF7,00-Norte Grande,NaN,NaN,NaN,PV,92.0,0.000000,0.000000,15,15.0,92.0,1,1
321,CCSS_LIKANANTAI_TRANSELEC,-,-,NaN,Generador Sincrónico,0,LF6,00-Norte Grande,NaN,NaN,NaN,PV,92.0,0.000000,0.000000,15,15.0,92.0,1,1
322,CCSS_ILLAPA_ALUPAR,-,-,NaN,Generador Sincrónico,0,LF4,01-Atacama,NaN,NaN,NaN,PV,270.0,0.000000,0.000000,15,15.0,270.0,1,1


# UNO DATA DE RESULTADOS

In [21]:
# uno la data de los resultados 
c=-1
N_tabla= len(Tabla_LF_GEN)
df = df_concatenado

for name1 in df_concatenado["Name1"]:
    c= c+1
    df = Tabla_LF_GEN 
    Tipo = df_concatenado.at[c,"Type"]  
    status = df_concatenado.at[c,"EMTP Status"]  
    if Tipo =="Parque Fotovoltaico":
        for i in range(N_tabla):
            if name1 == Tabla_LF_GEN.at[i,"Name1"] and status == 1:
                df_concatenado.at[c,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[i,"Tensión en Bornes [kV]"]
                df_concatenado.at[c,"P despachada [MW]"] = Tabla_LF_GEN.at[i,"Potencia Activa [MW]"]
                df_concatenado.at[c,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[i,"Potencia Reactiva [Mvar]"]
            else:
                continue
                   
    if Tipo =="Generador Sincrónico":
        nameLF=df_concatenado.at[c,"EMTP Load Flow Component"]
        for i in range(N_tabla):
            if nameLF == Tabla_LF_GEN.at[i,"NameLF"] and name1 ==Tabla_LF_GEN.at[i,"Name1"]:   
                df_concatenado.at[c,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[i,"Tensión en Bornes [kV]"]
                df_concatenado.at[c,"P despachada [MW]"] = Tabla_LF_GEN.at[i,"Potencia Activa [MW]"]
                df_concatenado.at[c,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[i,"Potencia Reactiva [Mvar]"]
            else : 
                continue        
    if Tipo =="Parque Eólico":
        for i in range(N_tabla):
            if name1 == Tabla_LF_GEN.at[i,"Name1"]:
                df_concatenado.at[c,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[i,"Tensión en Bornes [kV]"]
                df_concatenado.at[c,"P despachada [MW]"] = Tabla_LF_GEN.at[i,"Potencia Activa [MW]"]
                df_concatenado.at[c,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[i,"Potencia Reactiva [Mvar]"]     
            else : 
                continue      

    
      
        #df_concatenado.at[c,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[fila,"Tensión en Bornes [kV]"]
        #df_concatenado.at[c,"P despachada [MW]"] = Tabla_LF_GEN.at[fila,"Potencia Activa [MW]"]
        #df_concatenado.at[c,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[fila,"Potencia Reactiva [Mvar]"]


pd.set_option('display.max_rows', 250)
df_concatenado 

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PFV_Capricornio,PFV_Capricornio_new,NaN,PFV CAPRICORNIO,Parque Fotovoltaico,1,NaN,00-Norte Grande,105.74,60.01,-21.67,NaN,95.019,95.019000,-21.673834,NaN,110.0,95.019,57.0,57.0
1,PFV_Quilapilun,PFV_Quilapilun,NaN,PFV QUILAPILUN,Parque Fotovoltaico,1,NaN,03-Chilquinta-Aconcagua,227.17,99.22,-1.5,NaN,110.022,99.219840,-1.500300,NaN,220.0,110.022,66.0,66.0
2,PFV_Dona_Carmen,PFV_Dona_Carmen,NaN,PFV DOÑA CARMEN SOLAR,Parque Fotovoltaico,1,NaN,02-Coquimbo,226.04,27.94,-0.0,NaN,35.007,27.938920,0.000000,NaN,220.0,35.007,21.0,21.0
3,PFV_EL_PELICANO,PFV_EL_PELICANO,NaN,NaN,Parque Fotovoltaico,1,NaN,02-Coquimbo,226.54,94.5,-16.1,NaN,110.0022,67.871357,-16.104322,NaN,220.0,110.0022,66.0,66.0
4,PFV_LA_HUELLA,PFV_LA_HUELLA,NaN,PFV LA HUELLA,Parque Fotovoltaico,1,NaN,02-Coquimbo,226.46,79.4,-24.07,NaN,94.38,58.232460,-24.066900,NaN,220.0,94.38,26.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,CCSS_AMARIA_ALUPAR,-,-,NaN,Generador Sincrónico,0,LF5,00-Norte Grande,NaN,NaN,NaN,PV,360.0,0.000000,0.000000,15,15.0,360.0,1,1
320,CCSS_AMARIA_TRANSELEC,-,-,NaN,Generador Sincrónico,0,LF7,00-Norte Grande,NaN,NaN,NaN,PV,92.0,0.000000,0.000000,15,15.0,92.0,1,1
321,CCSS_LIKANANTAI_TRANSELEC,-,-,NaN,Generador Sincrónico,0,LF6,00-Norte Grande,NaN,NaN,NaN,PV,92.0,0.000000,0.000000,15,15.0,92.0,1,1
322,CCSS_ILLAPA_ALUPAR,-,-,NaN,Generador Sincrónico,0,LF4,01-Atacama,NaN,NaN,NaN,PV,270.0,0.000000,0.000000,15,15.0,270.0,1,1


# AGREGO GFM y STATCOM 

In [22]:
GFM_STAT = pd.read_excel('Plantas.xlsx',sheet_name="GFM_STAT" )
data_pre_final = df_concatenado
data_final = pd.concat([data_pre_final,GFM_STAT ], axis=0, ignore_index=True)


# ESCRibo excel 

In [23]:

with pd.ExcelWriter('Resultados/Data_SEN_target.xlsx') as writer:

    data_final.to_excel(writer, sheet_name='Data',index=False)
    #df_nuevo.to_excel(writer, sheet_name='Data',index=False)
 

In [75]:

# AGREGO GFM Y STATCOM

N_filas_add=0

for name1 in Tabla_LF_GEN["Name1"]:
    if "GFM" in name1 or "STAT" in name1 :
        N_filas_add=N_filas_add+1
for i in range(N_filas_add):
    df_concatenado.loc[len(df_concatenado)] = [None] * len(df_concatenado.columns)

N_data = len(df_concatenado)
N_data= N_data-N_filas_add 

C:\Users\fabian.medina\AppData\Local\Temp\ipykernel_11688\3462709706.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concatenado.loc[len(df_concatenado)] = [None] * len(df_concatenado.columns)
C:\Users\fabian.medina\AppData\Local\Temp\ipykernel_11688\3462709706.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concatenado.loc[len(df_concatenado)] = [None] * len(df_concatenado.columns)
C:\Users\fabian.medina\AppData\Local\Temp\ipykernel_11688\3462709706.py:9: FutureWarning: The behavi

In [76]:




c=-1
i=-1
for name1 in Tabla_LF_GEN["Name1"]:
    c=c+1
    if "GFM" in name1 :
        i=i+1
        df_concatenado.at[N_data+i,"Name1"]= name1
        df_concatenado.at[N_data+i,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[c,"Tensión en Bornes [kV]"]
        df_concatenado.at[N_data+i,"P despachada [MW]"] = Tabla_LF_GEN.at[c,"Potencia Activa [MW]"]
        df_concatenado.at[N_data+i,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[c,"Potencia Reactiva [Mvar]"]
        df_concatenado.at[N_data+i,"Type"] = "Grid Forming"
        df_concatenado.at[N_data+i,"EMTP Status"] = 1
    if "STAT" in name1:
        i=i+1
        df_concatenado.at[N_data+i,"Name1"]= name1
        df_concatenado.at[N_data+i,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[c,"Tensión en Bornes [kV]"]
        df_concatenado.at[N_data+i,"P despachada [MW]"] = Tabla_LF_GEN.at[c,"Potencia Activa [MW]"]
        df_concatenado.at[N_data+i,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[c,"Potencia Reactiva [Mvar]"]
        df_concatenado.at[N_data+i,"Type"] = "STATCOM"
        df_concatenado.at[N_data+i,"EMTP Status"] = 1
pd.set_option('display.max_rows', 250)        
df_concatenado

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PFV_Capricornio,PFV_Capricornio_new,NaN,PFV CAPRICORNIO,Parque Fotovoltaico,1,NaN,00-Norte Grande,105.74,60.01,-21.67,NaN,95.019,95.019000,-21.673834,NaN,110.0,95.019,57.0,57.0
1,PFV_Quilapilun,PFV_Quilapilun,NaN,PFV QUILAPILUN,Parque Fotovoltaico,1,NaN,03-Chilquinta-Aconcagua,227.17,99.22,-1.5,NaN,110.022,99.219840,-1.500300,NaN,220.0,110.022,66.0,66.0
2,PFV_Dona_Carmen,PFV_Dona_Carmen,NaN,PFV DOÑA CARMEN SOLAR,Parque Fotovoltaico,1,NaN,02-Coquimbo,226.04,27.94,-0.0,NaN,35.007,27.938920,0.000000,NaN,220.0,35.007,21.0,21.0
3,PFV_EL_PELICANO,PFV_EL_PELICANO,NaN,NaN,Parque Fotovoltaico,1,NaN,02-Coquimbo,226.54,94.5,-16.1,NaN,110.0022,67.871357,-16.104322,NaN,220.0,110.0022,66.0,66.0
4,PFV_LA_HUELLA,PFV_LA_HUELLA,NaN,PFV LA HUELLA,Parque Fotovoltaico,1,NaN,02-Coquimbo,226.46,79.4,-24.07,NaN,94.38,58.232460,-24.066900,NaN,220.0,94.38,26.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,ILLAPA_GFM,None,None,None,Grid Forming,1,None,None,0.59,100.0,9.45,None,None,NaN,NaN,None,None,None,None,None
322,LIKANA_GFM,None,None,None,Grid Forming,1,None,None,0.6,100.0,50.06,None,None,NaN,NaN,None,None,None,None,None
323,NCHUQUI_GFM,None,None,None,Grid Forming,1,None,None,0.6,100.0,44.24,None,None,NaN,NaN,None,None,None,None,None
324,AMARIA_GFM,None,None,None,Grid Forming,1,None,None,0.6,100.0,32.5,None,None,NaN,NaN,None,None,None,None,None


In [77]:
#valores nominales GFM Y STAT
df_GFM = pd.DataFrame({
    "Name1": ['ILLAPA_GFM','LIKANA_GFM','NCHUQUI_GFM','AMARIA_GFM'],
    'Nº_unid': [200,200,200,200 ],
    'Nº_unid_serv':[200,200,200,200 ],
    'Snom_uni [MVA]':[1,1,1,1],
    'Tensión Nominal [kV]': [0.575,0.575,0.575,0.575],
    'Sbase [MVA]': [200,200,200,200],
    'Potencia Nominal [MVA]': [200,200,200,200],
})

	
df_STAT = pd.DataFrame({
    "Name1": ['STAT_PAZUCAR','STAT_POLPAICO','STAT_CNAVIA'],
    'Nº_unid': [100,65,102],
    'Nº_unid_serv':[100,65,102],
    'Snom_uni [MVA]':[1,1,1],
    'Tensión Nominal [kV]': [0.575,0.575,0.575],
    'Sbase [MVA]': [100,65,102],
    'Potencia Nominal [MVA]': [100,65,102],
})

c=-1
for name1 in df_GFM["Name1"]:
    c=c+1
    print(name1)
    df = df_concatenado
    indice = df.index[df['Name1'] == name1].tolist()[0]
    df_concatenado.at[indice,"Nº_unid"] =  df_GFM.at[c,"Nº_unid"]
    df_concatenado.at[indice,"Nº_unid_serv"] =  df_GFM.at[c,"Nº_unid_serv"]
    df_concatenado.at[indice,"Tensión Nominal [kV]"] =  df_GFM.at[c,"Tensión Nominal [kV]"]
    df_concatenado.at[indice,"Sbase [MVA]"]=  df_GFM.at[c,"Sbase [MVA]"]
    print(indice,df_GFM.at[c,"Sbase [MVA]"])
    df_concatenado.at[indice,"Potencia Nominal [MVA]"] =  df_GFM.at[c,"Potencia Nominal [MVA]"]
c=-1    
for name1 in df_STAT["Name1"]:
    c=c+1
    print(name1)
    df = df_concatenado
    indice = df.index[df['Name1'] == name1].tolist()[0]
    df_concatenado.at[indice,"Nº_unid"] =  df_STAT.at[c,"Nº_unid"]
    df_concatenado.at[indice,"Nº_unid_serv"] =  df_STAT.at[c,"Nº_unid_serv"]
    df_concatenado.at[indice,"Tensión Nominal [kV]"] =  df_STAT.at[c,"Tensión Nominal [kV]"]
    df_concatenado.at[indice,"Sbase [MVA]"]=  df_STAT.at[c,"Sbase [MVA]"]
    df_concatenado.at[indice,"Potencia Nominal [MVA]"] =  df_STAT.at[c,"Potencia Nominal [MVA]"]
        
    
df_concatenado

ILLAPA_GFM
321 200
LIKANA_GFM
322 200
NCHUQUI_GFM
323 200
AMARIA_GFM
324 200
STAT_PAZUCAR
STAT_POLPAICO
STAT_CNAVIA


,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PFV_Capricornio,PFV_Capricornio_new,NaN,PFV CAPRICORNIO,Parque Fotovoltaico,1,NaN,00-Norte Grande,105.74,60.01,-21.67,NaN,95.019,95.019000,-21.673834,NaN,110.0,95.019,57.0,57.0
1,PFV_Quilapilun,PFV_Quilapilun,NaN,PFV QUILAPILUN,Parque Fotovoltaico,1,NaN,03-Chilquinta-Aconcagua,227.17,99.22,-1.5,NaN,110.022,99.219840,-1.500300,NaN,220.0,110.022,66.0,66.0
2,PFV_Dona_Carmen,PFV_Dona_Carmen,NaN,PFV DOÑA CARMEN SOLAR,Parque Fotovoltaico,1,NaN,02-Coquimbo,226.04,27.94,-0.0,NaN,35.007,27.938920,0.000000,NaN,220.0,35.007,21.0,21.0
3,PFV_EL_PELICANO,PFV_EL_PELICANO,NaN,NaN,Parque Fotovoltaico,1,NaN,02-Coquimbo,226.54,94.5,-16.1,NaN,110.0022,67.871357,-16.104322,NaN,220.0,110.0022,66.0,66.0
4,PFV_LA_HUELLA,PFV_LA_HUELLA,NaN,PFV LA HUELLA,Parque Fotovoltaico,1,NaN,02-Coquimbo,226.46,79.4,-24.07,NaN,94.38,58.232460,-24.066900,NaN,220.0,94.38,26.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,ILLAPA_GFM,None,None,None,Grid Forming,1,None,None,0.59,100.0,9.45,None,200,NaN,NaN,None,0.575,200,200,200
322,LIKANA_GFM,None,None,None,Grid Forming,1,None,None,0.6,100.0,50.06,None,200,NaN,NaN,None,0.575,200,200,200
323,NCHUQUI_GFM,None,None,None,Grid Forming,1,None,None,0.6,100.0,44.24,None,200,NaN,NaN,None,0.575,200,200,200
324,AMARIA_GFM,None,None,None,Grid Forming,1,None,None,0.6,100.0,32.5,None,200,NaN,NaN,None,0.575,200,200,200


# ESCRIBO EXCEL

In [78]:


with pd.ExcelWriter('Resultados/Data_SEN_target.xlsx') as writer:

    df_concatenado.to_excel(writer, sheet_name='Data',index=False)
    #df_nuevo.to_excel(writer, sheet_name='Data',index=False)
 